We'll start by importing the modules required to complete this project
psycopg2 - To create a connection with the database and run SQL scripts
Faker - To generate fake data for the database
Random - Used with faker to generate random data

In [1]:
import psycopg2
from faker import Faker
import random
import json
import datetime


Task 1: Database Setup
Create the database and tables using appropriate data types and constraints and populate the table


In [ ]:

try:
    conn = psycopg2.connect(
        host = "localhost",
        user = "yinka_admin",
        password = "yinkaadmin",
        dbname = "postgres"
    )
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()

    cur.execute("DROP DATABASE IF EXISTS exam_db")
    conn.commit()

    cur.execute(f"CREATE DATABASE exam_db")
    conn.commit()

    print(f"Database exam_db created successfully")

except (Exception, psycopg2.Error) as error:
    print(f"Error while creating database: {error}")

finally:
    if conn:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed.")



In [ ]:
# Connect to the database you created
try:
    conn = psycopg2.connect(
        host = "localhost",
        user = "yinka_admin",
        password = "yinkaadmin",
        dbname = "exam_db"
    )
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()

    cur.execute("CREATE SCHEMA IF NOT EXISTS exam_schema")
    conn.commit()
    print("Exam schema successfully created")


except (Exception, psycopg2.Error) as error:
    print(f"Error while creating schema: {error}")

In [ ]:
# Create customer table
try:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS customers(
                customer_id SERIAL PRIMARY KEY,
                name TEXT NOT NULL,
                email TEXT UNIQUE NOT NULL,
                phone_number TEXT,
                address JSONB)
                """
                )

    # Create product table
    cur.execute("""
                CREATE TABLE IF NOT EXISTS products(
                product_id SERIAL PRIMARY KEY,
                product_name VARCHAR(100) NOT NULL,
                category VARCHAR(100),
                price DECIMAL(10, 2) NOT NULL,
                stock_quantity INT NOT NULL 
                )
                """)

    # Create the orders table
    cur.execute("""
                CREATE TABLE IF NOT EXISTS orders(
                order_id SERIAL PRIMARY KEY,
                customer_id INT NOT NULL, 
                order_date TIMESTAMP NOT NULL,
                total_amount DECIMAL(10, 2) NOT NULL,
                FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
                )
                """)

    # Create the order item table
    cur.execute("""
                CREATE TABLE IF NOT EXISTS order_items(
                order_item_id SERIAL PRIMARY KEY,
                order_id INT NOT NULL,
                product_id INT NOT NULL,
                quantity INT NOT NULL,
                price DECIMAL(10,2) NOT NULL,
                FOREIGN KEY (order_id) REFERENCES orders(order_id),
                FOREIGN KEY (product_id) REFERENCES productS(product_id)
                )
                """)
    
    conn.commit()
    print("Tables created successfully")

except (Exception, psycopg2.Error) as error:
    print(f"Error while creating table {error}")


In [ ]:
# Populate the tables with data
fake = Faker()

# Generate customer data
cur.execute("TRUNCATE TABLE customers, products, orders, order_items")
print("Tables truncated successfully")

customers = [
        ("John Doe", "john.doe@example.com", "555-1234", '{"Street": "123 Main St", "City": "Anytown", "State": "CA", "PostalCode": "12345"}'),
        ("Jane Smith", "jane.smith@example.com", "555-5678", '{"Street": "456 Oak Ave", "City": "Smallville", "State": "NY", "PostalCode": "56789"}'),
        ("David Lee", "david.lee@example.com", "555-9012", '{"Street": "789 Pine Rd", "City": "Big City", "State": "TX", "PostalCode": "90123"}'),
        ("Sarah Jones", "sarah.jones@example.com", "555-3456", '{"Street": "101 Maple Dr", "City": "Othertown", "State": "FL", "PostalCode": "34567"}'),
        ("Michael Brown", "michael.brown@example.com", "555-7890", '{"Street": "202 Elm St", "City": "Anytown", "State": "CA", "PostalCode": "12345"}'),
        ("Emily Davis", "emily.davis@example.com", "555-2345", '{"Street": "303 Oak Ave", "City": "Smallville", "State": "NY", "PostalCode": "56789"}'),
        ("Daniel White", "daniel.white@example.com", "555-6789", '{"Street": "404 Pine Rd", "City": "Big City", "State": "TX", "PostalCode": "90123"}'),
        ("Amanda Clark", "amanda.clark@example.com", "555-8901", '{"Street": "505 Maple Dr", "City": "Othertown", "State": "FL", "PostalCode": "34567"}'),
        ("Christopher Lee", "christopher.lee@example.com", "555-0123", '{"Street": "606 Elm St", "City": "Anytown", "State": "CA", "PostalCode": "12345"}'),
        ("Melissa Young", "melissa.young@example.com", "555-4567", '{"Street": "707 Oak Ave", "City": "Smallville", "State": "NY", "PostalCode": "56789"}')
    ]
for name, email, phone, address in customers:
    cur.execute("INSERT INTO customers (name, email, phone_number, address) VALUES (%s, %s, %s, %s)", 
                (name, email, phone, address))
conn.commit()

cur.execute("SELECT * FROM customers")
customer_table = cur.fetchall()
for row in customer_table:
     print(row)



In [ ]:
#Generate product data

products = [
        ("Acme Corp", "Electronics", 999.00, 50),
        ("Beta Inc.", "Clothing", 49.99, 20),
        ("Gamma Co.", "Books", 29.95, 100),
        ("Delta Ltd.", "Food", 19.99, 50),
        ("Epsilon Corp.", "Home", 149.00, 30),
        ("Zeta Inc.", "Electronics", 299.00, 75),
        ("Alpha Corp.", "Clothing", 79.99, 40),
        ("Omega Inc.", "Books", 14.95, 150),
        ("Theta Co.", "Food", 9.99, 100),
        ("Lambda Corp.", "Home", 249.00, 25)
    ]
for name, category, price, stock_quantity in products:
        cur.execute("INSERT INTO products (product_name, category, price, stock_quantity) VALUES (%s, %s, %s, %s)", 
                    (name, category, price, stock_quantity))
conn.commit()


cur.execute("SELECT * FROM products")
customer_table = cur.fetchall()
for row in customer_table:
     print(row)

In [ ]:
# Generate order data

orders = [
        (1, 1234.56, datetime.date(2023, 11, 22)), 
        (3, 456.78, datetime.date(2023, 11, 21)), 
        (2, 987.65, datetime.date(2023, 11, 20)), 
        (5, 234.56, datetime.date(2023, 11, 19)), 
        (4, 678.90, datetime.date(2023, 11, 18)), 
        (1, 1011.22, datetime.date(2023, 11, 17)), 
        (3, 345.67, datetime.date(2023, 11, 16)), 
        (2, 876.54, datetime.date(2023, 11, 15)), 
        (5, 432.10, datetime.date(2023, 11, 14)), 
        (4, 987.65, datetime.date(2023, 11, 13)), 
        (1, 567.89, datetime.date(2023, 11, 12)), 
        (3, 123.45, datetime.date(2023, 11, 11)), 
        (2, 789.01, datetime.date(2023, 11, 10)), 
        (5, 345.67, datetime.date(2023, 11, 9)), 
        (4, 901.23, datetime.date(2023, 11, 8)), 
        (1, 567.89, datetime.date(2023, 11, 7)), 
        (3, 123.45, datetime.date(2023, 11, 6)), 
        (2, 789.01, datetime.date(2023, 11, 5)), 
        (5, 345.67, datetime.date(2023, 11, 4)), 
        (4, 901.23, datetime.date(2023, 11, 3))
    ]
for customer_id, total_amount, order_date in orders:
    cur.execute("INSERT INTO orders (customer_id, total_amount, order_date) VALUES (%s, %s, %s)", 
                (customer_id, total_amount, order_date))
conn.commit()

cur.execute("SELECT * FROM orders")
customer_table = cur.fetchall()
for row in customer_table:
     print(row)

In [ ]:
# Generate order item data

order_items = [
        (1, 1, 2, 999.00), 
        (1, 1, 1, 29.95), 
        (2, 2, 3, 49.99), 
        (2, 5, 1, 149.00), 
        (3, 1, 1, 999.00), 
        (3, 4, 2, 19.99), 
        (4, 3, 1, 29.95), 
        (4, 2, 1, 49.99), 
        (5, 1, 1, 999.00), 
        (5, 5, 1, 149.00), 
        (6, 1, 2, 999.00), 
        (6, 3, 1, 29.95), 
        (7, 2, 3, 49.99), 
        (7, 5, 1, 149.00), 
        (8, 1, 1, 999.00), 
        (8, 4, 2, 19.99), 
        (9, 3, 1, 29.95), 
        (9, 2, 1, 49.99), 
        (10, 1, 1, 999.00), 
        (10, 5, 1, 149.00), 
        (11, 1, 2, 999.00), 
        (11, 1, 1, 29.95), 
        (12, 2, 3, 49.99), 
        (12, 5, 1, 149.00), 
        (13, 1, 1, 999.00), 
        (13, 4, 2, 19.99), 
        (14, 3, 1, 29.95), 
        (14, 2, 1, 49.99), 
        (15, 1, 1, 999.00), 
        (15, 5, 1, 149.00), 
        (16, 1, 2, 999.00), 
        (16, 3, 1, 29.95), 
        (17, 2, 3, 49.99), 
        (17, 5, 1, 149.00), 
        (18, 1, 1, 999.00), 
        (18, 4, 2, 19.99), 
        (19, 3, 1, 29.95), 
        (19, 2, 1, 49.99), 
        (20, 1, 1, 999.00), 
        (20, 5, 1, 149.00), 
        (1, 1, 2, 999.00), 
        (1, 1, 1, 29.95), 
        (2, 2, 3, 49.99), 
        (2, 5, 1, 149.00), 
        (3, 1, 1, 999.00), 
        (3, 4, 2, 19.99), 
        (4, 3, 1, 29.95), 
        (4, 2, 1, 49.99), 
        (5, 1, 1, 999.00), 
        (5, 5, 1, 149.00) 
    ]
for order_id, product_id, quantity, price in order_items:
    cur.execute("INSERT INTO order_items (order_id, product_id, quantity, price) VALUES (%s, %s, %s, %s)", 
                (order_id, product_id, quantity, price))

conn.commit()

cur.execute("SELECT * FROM orders")
customer_table = cur.fetchall()
for row in customer_table:
     print(row)

print("All Table data generated successfully.")

Task 2 - CRUD Operations

In [ ]:
# 1. Add a new customer to the database.
try:
    cur.execute("INSERT INTO customers (name, email, phone_number, address) VALUES (%s, %s, %s, %s)", 
                    ("Adeyinka", "owodunniadeyik@gmail.com", +2341023456, json.dumps({"Street": "50 Wiga", "City": "Mushin", "State": "Lagos", "PostalCode": "1200071"})))
    print("Data inserted successfully")

    conn.commit()

except (Exception, psycopg2.Error) as error:
    print(f"Error while inserting data {error}")



In [15]:
# 2. Update the stock quantity of a product after a purchase.
try:
    cur.execute("""
                UPDATE products
                SET stock_quantity = 50
                WHERE product_id = 6
""")
    
    conn.commit()
    print("Data Updated successfully")

except (Exception, psycopg2.Error) as error:
    print(f"Error while updating data {error}")



Data Updated successfully


In [16]:
# 3. Delete an order from the database.

try:
    cur.execute("DELETE FROM order_items WHERE order_id = 5")
    conn.commit()
    cur.execute("""
                DELETE FROM orders
                WHERE order_id = 5
""")
    conn.commit()
    print("Data deleted successfully")

except (Exception, psycopg2.Error) as error:
    print(f"Error while deleting data {error}")


Data deleted successfully


In [17]:
# 4. Retrieve all orders made by a specific customer.
try:
    cur.execute("""
        SELECT o.order_id, o.order_date, o.total_amount 
        FROM orders o
        JOIN customers c ON o.customer_id = c.customer_id
        WHERE c.customer_id = 4;
""")
    print("Data retrieved successfully")
    orders = cur.fetchall()
    for rows in orders:
        print(orders)

except (Exception, psycopg2.Error) as error:
    print(f"Error while retrieving data {error}")


Data retrieved successfully
[(10, datetime.datetime(2023, 11, 13, 0, 0), Decimal('987.65')), (15, datetime.datetime(2023, 11, 8, 0, 0), Decimal('901.23')), (20, datetime.datetime(2023, 11, 3, 0, 0), Decimal('901.23'))]
[(10, datetime.datetime(2023, 11, 13, 0, 0), Decimal('987.65')), (15, datetime.datetime(2023, 11, 8, 0, 0), Decimal('901.23')), (20, datetime.datetime(2023, 11, 3, 0, 0), Decimal('901.23'))]
[(10, datetime.datetime(2023, 11, 13, 0, 0), Decimal('987.65')), (15, datetime.datetime(2023, 11, 8, 0, 0), Decimal('901.23')), (20, datetime.datetime(2023, 11, 3, 0, 0), Decimal('901.23'))]
